Установим нужные библиотеки

In [ ]:
import requests as rq
from bs4 import BeautifulSoup, NavigableString

Создаем функцию, которая собирает нужные ссылки с сайта для дальнейшего парсинга

In [ ]:
def parse_page_with_articles(url): # собираем нужные ссылки на статьи
    page = rq.get(url)
    soup = BeautifulSoup(page.text, features="html.parser")

    urls = []

    for link in soup.find_all('a'):
        urls.append(link.get('href'))

    releases = []

    for link in soup.find_all('a'):
        if 'radosti-zhizni' in link.get('href') and link.get('href') not in releases:
            releases.append(link.get('href'))

    realeases_links = []
    page_links = []
    black_list = ['/radosti-zhizni', '/radosti-zhizni/motivatsiya', '/radosti-zhizni/otnosheniya',
                  '/radosti-zhizni/transformatsiya']
    for link in releases:
        if 'start' in link:
            page_links.append('https://shape.ru' + link)
        elif link not in black_list:
            realeases_links.append('https://shape.ru' + link)

    return realeases_links, page_links

Функция для того, чтобы достать нужные данные со страницы

In [ ]:
def GetArticle(url0):
    page0 = rq.get(url0)
    soup0 = BeautifulSoup(page0.text, features="html.parser")
    title = soup0.find_all('meta', {'property': 'og:title'})[0].attrs['content']
    text = soup0.find_all('meta', {'property': 'og:description'})[0].attrs['content']

    final_text = text.strip().replace('\xa0', ' ')
    return url0, title, final_text

Парсим страницы

In [ ]:
pages = ['https://shape.ru/radosti-zhizni']
looked_pages = []
all_articles_urls = []

counter = 0

while pages:  
    page = pages.pop(0)  # Берем первую ссылку со статьями
    
    articles_urls, parsed_page_urls = parse_page_with_articles(page)
    all_articles_urls.extend(articles_urls)

    looked_pages.append(page)
    for p_url in parsed_page_urls:
        if p_url not in looked_pages and p_url not in pages:
            pages.append(p_url)



In [ ]:
print(all_articles_urls)

['https://shape.ru/radosti-zhizni/transformatsiya/5045-kak-izbezhat-fevralskoj-depressii-sovety-kotorye-rabotayut', 'https://shape.ru/radosti-zhizni/transformatsiya/5044-chto-nas-zhdet-v-god-krolika-osobennosti-obshcheniya', 'https://shape.ru/radosti-zhizni/transformatsiya/5037-5-sovetov-kotorye-pomogut-nauchit-sya-dostigat-tselej-2', 'https://shape.ru/radosti-zhizni/transformatsiya/5036-esli-vy-trudogolik-kak-otuchit-sebya-rabotat-v-vykhodnye', 'https://shape.ru/radosti-zhizni/motivatsiya/5019-raj-dlya-dvoikh-7-napravlenij-dlya-puteshestviya-svidaniya', 'https://shape.ru/radosti-zhizni/transformatsiya/5018-5-sovetov-kotorye-pomogut-nauchit-sya-dostigat-tselej', 'https://shape.ru/radosti-zhizni/transformatsiya/5020-chto-v-zhizni-vazhnee-sem-ya-ili-kar-era', 'https://shape.ru/radosti-zhizni/transformatsiya/5011-daosskij-prognoz-na-god-vodyanogo-krolika', 'https://shape.ru/radosti-zhizni/transformatsiya/5003-minimalizm-otkaz-ot-nenuzhnykh-veshchej-kak-otkaz-ot-nenuzhnykh-problem', 'https

Берем только часть статей для работы

In [ ]:
counter = 0
articles = []
for link in all_articles_urls:
  try:
    new = GetArticle(link)
    articles.append(new)
    print(new)
    counter += 1
  except:
    pass

Собираем данные в датафрейм

In [ ]:
import pandas as pd
df = pd.DataFrame(articles)
df.head()

df.columns = ['link', 'title', 'text']
print(df.head(12))

df.to_csv('Корпус Shape.csv', encoding='utf-8') 


In [ ]:
df.tail()